In [1]:
import pandas as pd
from scipy.stats import pearsonr

In [63]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/data/ref/EPIC.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index

In [ ]:
%%time
wb=pd.read_csv('data/array/GSE132203_Geo_Submission_GTPEpic.csv',nrows=1000000,usecols=[0]+list(range(1,1591,2)),index_col=0).round(3)

In [ ]:
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE132nnn/GSE132203/matrix/GSE132203_series_matrix.txt.gz',skiprows=29+10,nrows=0,index_col=0).columns.str.strip('age: ').astype(float)

In [ ]:
%%time 
wb_t=wb.T
wb_t['age']=age
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

In [70]:
wb_cor.tail(100000)

cg08117436    0.097734
cg02133430    0.097734
cg20918563    0.097735
cg17374364    0.097735
cg20681223    0.097736
                ...   
cg22454769    0.695379
cg21572722    0.698479
cg17268658    0.717995
cg06639320    0.739374
cg16867657    0.794723
Length: 100000, dtype: float64

In [64]:
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')

In [56]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/arc/hmr/neo',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [57]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append(-1)
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append(-1)
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            l.append(-1)
            cur+=1

CPU times: user 40.3 s, sys: 476 ms, total: 40.8 s
Wall time: 40.8 s


In [65]:
man['l']=l
mg=pd.DataFrame(man.groupby('l')[0].count())
mg=mg[mg[0]>=4].copy()
print(len(mg))
mg['cor']=man.groupby('l')['cor'].mean()
hmr.iloc[mg.sort_values('cor').tail(10).index][[0,1,2]]

17119


,0,1,2
3,,,
HYPO1919,chr1,47416145,47417392
HYPO1709,chr1,40708854,40709944
HYPO24496,chr19,51887336,51888227
HYPO24766,chr19,58203918,58204641
HYPO31799,chr3,9552189,9554017
HYPO23941,chr19,44259825,44260516
HYPO23856,chr19,41996936,41999519
HYPO43087,chr8,56447897,56448292
HYPO1989,chr1,52842501,52844181


In [66]:
%%time
man=man[man['l']>0].copy()
man['bin']=man['cor']>0
cont=[]
c=0
last=False
last_l=-1
for i,b in man[['l','bin']].iterrows():
    if (b['bin']):
        if (b['l']>last_l or last==False):
            c+=1
            last_l=b['l']
        cont.append(c)
        last=True  
    else:
        cont.append(-1)
        last=False

CPU times: user 14.8 s, sys: 373 ms, total: 15.2 s
Wall time: 15.2 s


In [67]:
man['cont']=cont
mgc=pd.DataFrame(man.groupby('cont')[0].count())
mgc['chr']=man.groupby('cont')[0].first()
mgc['b']=man.groupby('cont')[1].first()
mgc['e']=man.groupby('cont')[1].last()
mgc['cor']=man.groupby('cont')['cor'].mean()
topl=mgc[mgc[0]>=5].sort_values('cor')
topl.tail(50)

,0,chr,b,e,cor
cont,,,,,
14290,8,chr5,141392455,141392944,0.309905
8425,8,chr3,9703726,9705498,0.310595
20144,25,chr8,24912675,24915754,0.310830
43184,12,chr19,56367999,56369018,0.311031
15296,6,chr6,12749119,12750159,0.311996
43193,6,chr19,56638067,56638261,0.312973
41587,9,chr19,19170159,19170664,0.313437
18912,5,chr7,100656277,100656631,0.313701
43267,10,chr19,58033753,58034468,0.314064


In [69]:
topl[topl['chr']=='chr6'].tail(20)

,0,chr,b,e,cor
cont,,,,,
16490,8,chr6,79947694,79948108,0.255219
16966,17,chr6,133241322,133242640,0.255752
16516,24,chr6,83707725,83709893,0.256725
15248,26,chr6,10380962,10386834,0.257685
16653,21,chr6,100453927,100459018,0.260857
17143,16,chr6,146543191,146544350,0.261603
17057,6,chr6,138795809,138796330,0.264402
16366,9,chr6,55579034,55579536,0.268595
16655,12,chr6,100467314,100469994,0.270832
